In [1]:
#import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard
import os, shutil
from PIL import Image, ImageDraw, ImageFont, ImageColor
import random
import pandas as pd
from sklearn.multioutput import RegressorChain

In [2]:
%load_ext tensorboard

In [3]:
train_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project GitHub\\Training_set_6_15' 
test_dataset_dir='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project GitHub\\Testing_set_6_16' 

df=pd.read_csv(os.path.join(train_dataset_dir,"training_dataframe.csv"))
df = df.sample(frac=1,random_state=5).reset_index(drop=True)

dft = pd.read_csv(os.path.join(test_dataset_dir,"training_dataframe.csv"))
dft = dft.sample(frac=1,random_state=5).reset_index(drop=True)

In [4]:
df.shape
#dft.shape

(121500, 9)

In [9]:
def build_model():
    model = models.Sequential()                                  
    model.add(layers.Conv2D(32, (3,3) ,activation='relu', input_shape=(32,32,1)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    return model

In [6]:
#Creates generators for training and validation images
SEED=5
validation_split_frac=0.2
batch_size=256
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=validation_split_frac)
test_datagen = ImageDataGenerator(rescale=1./255)  
train_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        color_mode='grayscale',
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='training')

validation_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        color_mode='grayscale',
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='validation')

# test_generator= test_datagen.flow_from_dataframe(
#         dft,
#         directory=test_dataset_dir,
#         x_col="filename",
#         y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
#         target_size=(32,32),
#         shuffle=True,
#         seed= SEED,
#         class_mode="raw",
#         batch_size=batch_size)

Found 97200 validated image filenames.
Found 24300 validated image filenames.


In [7]:
# from tensorboard import notebook
# notebook.list()
%tensorboard --logdir Training_logs/

Reusing TensorBoard on port 6006 (pid 3588), started 21:26:08 ago. (Use '!kill 3588' to kill it.)

In [10]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae','mape'])
#model.summary()
num_epochs=40
training_steps=int(df.shape[0]*(1-validation_split_frac)/batch_size)
validation_steps=int(df.shape[0]*validation_split_frac/batch_size)
#assert (training_steps+validation_steps)*batch_size == df.shape[0]
log_dir = os.path.join("Training_logs\\fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train_generator,
    steps_per_epoch=training_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=num_epochs,
    verbose=1, 
    callbacks=[tensorboard_callback]
    )


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 379 steps, validate for 94 steps
Epoch 1/40
379/379 [==============================] - 74s 195ms/step - loss: 0.0746 - mae: 0.2334 - mape: 1015043.3125 - val_loss: 0.0744 - val_mae: 0.2333 - val_mape: 1044194.0625
Epoch 2/40
379/379 [==============================] - 54s 141ms/step - loss: 0.0743 - mae: 0.2331 - mape: 1034787.5625 - val_loss: 0.0744 - val_mae: 0.2333 - val_mape: 1044190.8125
Epoch 3/40
379/379 [==============================] - 52s 138ms/step - loss: 0.0743 - mae: 0.2331 - mape: 1035580.9375 - val_loss: 0.0744 - val_mae: 0.2333 - val_mape: 1050863.7500
Epoch 4/40
379/379 [==============================] - 54s 142ms/step - loss: 0.0743 - mae: 0.2331 - mape: 1034889.5000 - val_loss: 0.0744 - val_mae: 0.2333 - val_mape: 1043696.0625
Epoch 5/40
379/379 [==============================] - 50s 132ms/step - loss: 0.0743 - mae: 0.2331 - mape: 1035784.2500 - val_loss: 0.0744 - val_mae: 0.2333 - val_mape: 1042259.5625
Ep

In [11]:
model.save("40erun6_17")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 40erun6_17\assets


In [19]:
model=models.load_model('60epochrun6_16')

In [22]:
#Evaluates the model on testing set
testing_steps=dft.shape[0]//batch_size
log_dir = os.path.join("Training_logs\\test", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.evaluate(
    test_generator,
    steps=testing_steps
    #callbacks=[tensorboard_callback]
)

  ...
    to  
  ['...']
42/42 [==============================] - 4s 91ms/step - loss: 0.0112 - mae: 0.0784


[0.011231104200262399, 0.078366056]

['loss', 'mae']

On the scale of two dimensional materials, stretching or straining the atomic lattice can have an enormous impact on the material properties of whatever you're working with. We can take photos of these materials with atomic resoution, however, the exact distribution of these atoms is hard to characterize by eye, and small strain of 1-2% is almost impossible to distinguish at first glance. My project is to design a machine learning model that can automaticaly detect the strain of a honeycomb lattice, even when that strain is small and varrying across an image. This would allow researchers to compare the strain of a material to its electronic properties and other observables, unveiling connections we haven't been able to see before. Many 2D materials are poorly understood, and this project could help answer many questions about how 2D phenomenon arise.